# Connection to blob

In [0]:
storageAccount = "xxxxxxxxxx"
containerName = "xxxxxxx"
mountpoint = "/mnt/xxxxxxx/"
storageEndpoint = f"wasbs://{containerName}@{storageAccount}.blob.core.windows.net" 
storageKey = dbutils.secrets.get(scope="xxxxxxx",key="xxxxxx")
storageConn = f"fs.azure.account.key.{storageAccount}.blob.core.windows.net"
try:
    if not any(mount.mountPoint==mountpoint for mount in dbutils.fs.mounts()):
        dbutils.fs.mount(
        source = storageEndpoint,
        mount_point = mountpoint,
        extra_configs = {storageConn:storageKey}
    )
        print(f"{mountpoint} has been mounted")
    else:
        print(f"Mount point '{mountpoint}' is already mounted.")
except Exception as e:
    raise e  # Re-raise the exception if mounting fails


Mount point '/mnt/ingrid-sollim/' is already mounted.


In [0]:
path=mountpoint+"bikes"
display(dbutils.fs.ls(path))

path,name,size,modificationTime
dbfs:/mnt/ingrid-sollim/bikes/customer.csv,customer.csv,1813963,1694474860000
dbfs:/mnt/ingrid-sollim/bikes/person.csv,person.csv,13646947,1694474860000
dbfs:/mnt/ingrid-sollim/bikes/product.csv,product.csv,104823,1694474860000
dbfs:/mnt/ingrid-sollim/bikes/salesorderdetail.csv,salesorderdetail.csv,13801182,1694474860000
dbfs:/mnt/ingrid-sollim/bikes/salesorderheader.csv,salesorderheader.csv,8267704,1694474862000
dbfs:/mnt/ingrid-sollim/bikes/spoffer.csv,spoffer.csv,36680,1694474860000


In [0]:
df = spark.read.option("header",True).options(delimiter=";").csv(path+"/customer.csv")
display(df.limit(10))

CustomerID,PersonID,StoreID,TerritoryID,AccountNumber,rowguid,ModifiedDate
1,NULL,934,1,AW00000001,3F5AE95E-B87D-4AED-95B4-C3797AFCB74F,2014-09-12 11:15:07.263
2,NULL,1028,1,AW00000002,E552F657-A9AF-4A7D-A645-C429D6E02491,2014-09-12 11:15:07.263
3,NULL,642,4,AW00000003,130774B1-DB21-4EF3-98C8-C104BCD6ED6D,2014-09-12 11:15:07.263
4,NULL,932,4,AW00000004,FF862851-1DAA-4044-BE7C-3E85583C054D,2014-09-12 11:15:07.263
5,NULL,1026,4,AW00000005,83905BDC-6F5E-4F71-B162-C98DA069F38A,2014-09-12 11:15:07.263
6,NULL,644,4,AW00000006,1A92DF88-BFA2-467D-BD54-FCB9E647FDD7,2014-09-12 11:15:07.263
7,NULL,930,1,AW00000007,03E9273E-B193-448E-9823-FE0C44AEED78,2014-09-12 11:15:07.263
8,NULL,1024,5,AW00000008,801368B1-4323-4BFA-8BEA-5B5B1E4BD4A0,2014-09-12 11:15:07.263
9,NULL,620,5,AW00000009,B900BB7F-23C3-481D-80DA-C49A5BD6F772,2014-09-12 11:15:07.263
10,NULL,928,6,AW00000010,CDB6698D-2FF1-4FBA-8F22-60AD1D11DABD,2014-09-12 11:15:07.263


#Check for column changes

In [0]:
def check_column_changes(df, columns):
    '''This function checks if new columns had been added or if the column names had changed'''
    #Get the column names from the dataframe
    current_columns = df.columns
    # Check if the column names have changed.
    for columns, current_columns in zip(old_columns, new_columns):
        if old_column != new_column:
            print("There are differences in the code")
            return True

In [0]:
#column_names=customer_df.columns
column_names = ['CustomerID','PersonID','StoreID','TerritoryID','AccountNumber','rowguid','ModifiedDate']


Out[29]: ['CustomerID',
 'PersonID',
 'StoreID',
 'TerritoryID',
 'AccountNumber',
 'rowguid',
 'ModifiedDate']

#Check for missing data

#Change Data types

In [0]:
df.printSchema()

root
 |-- CustomerID: string (nullable = true)
 |-- PersonID: string (nullable = true)
 |-- StoreID: string (nullable = true)
 |-- TerritoryID: string (nullable = true)
 |-- AccountNumber: string (nullable = true)
 |-- rowguid: string (nullable = true)
 |-- ModifiedDate: string (nullable = true)



In [0]:
from pyspark.sql.functions import col, to_timestamp

customer_df = df.withColumn("CustomerID",col("CustomerID").cast("int"))\
.withColumn("StoreID",col("StoreID").cast("int"))\
.withColumn("TerritoryID",col("TerritoryID").cast("int"))\
.withColumn("ModifiedDate",to_timestamp(col("ModifiedDate"),"yyyy-MM-dd HH:mm:ss.SSS"))

display(customer_df.limit(10))

CustomerID,PersonID,StoreID,TerritoryID,AccountNumber,rowguid,ModifiedDate
1,NULL,934,1,AW00000001,3F5AE95E-B87D-4AED-95B4-C3797AFCB74F,2014-09-12T11:15:07.263+0000
2,NULL,1028,1,AW00000002,E552F657-A9AF-4A7D-A645-C429D6E02491,2014-09-12T11:15:07.263+0000
3,NULL,642,4,AW00000003,130774B1-DB21-4EF3-98C8-C104BCD6ED6D,2014-09-12T11:15:07.263+0000
4,NULL,932,4,AW00000004,FF862851-1DAA-4044-BE7C-3E85583C054D,2014-09-12T11:15:07.263+0000
5,NULL,1026,4,AW00000005,83905BDC-6F5E-4F71-B162-C98DA069F38A,2014-09-12T11:15:07.263+0000
6,NULL,644,4,AW00000006,1A92DF88-BFA2-467D-BD54-FCB9E647FDD7,2014-09-12T11:15:07.263+0000
7,NULL,930,1,AW00000007,03E9273E-B193-448E-9823-FE0C44AEED78,2014-09-12T11:15:07.263+0000
8,NULL,1024,5,AW00000008,801368B1-4323-4BFA-8BEA-5B5B1E4BD4A0,2014-09-12T11:15:07.263+0000
9,NULL,620,5,AW00000009,B900BB7F-23C3-481D-80DA-C49A5BD6F772,2014-09-12T11:15:07.263+0000
10,NULL,928,6,AW00000010,CDB6698D-2FF1-4FBA-8F22-60AD1D11DABD,2014-09-12T11:15:07.263+0000


In [0]:
customer_df.printSchema()

root
 |-- CustomerID: integer (nullable = true)
 |-- PersonID: string (nullable = true)
 |-- StoreID: integer (nullable = true)
 |-- TerritoryID: integer (nullable = true)
 |-- AccountNumber: string (nullable = true)
 |-- rowguid: string (nullable = true)
 |-- ModifiedDate: timestamp (nullable = true)



In [0]:
#Rename columns
customer_df = customer_df.withColumnRenamed("CustomerID","customer_id")\
    .withColumnRenamed("PersonID","person_id").withColumnRenamed("StoreID","store_id")\
    .withColumnRenamed("TerritoryID","territory_id").withColumnRenamed("AccountNumber","account_number").withColumnRenamed("ModifiedDate","modified_date")



#Ingest the cleaned data to the database

### Connection to the database

In [0]:
# Get the secrets from the jdbc secret scope.
username = dbutils.secrets.get(scope="xxxxxxx", key="xxxxxx")
password = dbutils.secrets.get(scope="xxxxxxx", key="xxxxxxx")


jdbcUsername = username
jdbcPassword = password
jdbcHostname = "srv-fab-projetos.database.windows.net"
jdbcPort = 1433
jdbcDatabase = "xxxxxx"

jdbcUrl = "jdbc:sqlserver://{0}:{1};database={2}".format(jdbcHostname, jdbcPort, jdbcDatabase)

connectionProperties = {
  "user" : jdbcUsername,
  "password" : jdbcPassword,
  "driver" : "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

table_name = 'xxxxxx'

In [0]:
customer_df.write \
  .format("jdbc") \
  .option("url", jdbcUrl) \
  .option("dbtable", table_name) \
  .mode("overwrite") \
  .options(**connectionProperties) \
  .save()

### Insert into the database


In [0]:
dbutils.fs.unmount(mountpoint)

/mnt/ingrid-sollim/ has been unmounted.
Out[5]: True